### https://medium.com/coinmonks/a-box-detection-algorithm-for-any-image-containing-boxes-756c15d7ed26

# Importing Modules

In [ ]:
import cv2
import numpy as np
import argparse
import imutils
import os
import matplotlib.pyplot as plt

# Defining the table for Croping Characters

In [ ]:
# Read the image
img = cv2.imread("A.jpeg", 0)

# Thresholding the image
(thresh, img_bin) = cv2.threshold(img, 128, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# Invert the image
img_bin = 255-img_bin 
#cv2.imwrite("Image_bin.jpg",img_bin)

In [ ]:
# Defining a kernel length
kernel_length = np.array(img).shape[1]//80
 
# A verticle kernel of (1 X kernel_length), which will detect all the verticle lines from the image.
verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
# A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
# A kernel of (3 X 3) ones.
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

In [ ]:
# Morphological operation to detect vertical lines from an image
img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)
verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
##cv2.imwrite("verticle_lines.jpg",verticle_lines_img)
# Morphological operation to detect horizontal lines from an image
img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)
##cv2.imwrite("horizontal_lines.jpg",horizontal_lines_img)

In [ ]:
# Weighting parameters, this will decide the quantity of an image to be added to make a new image.
alpha = 0.5
beta = 1.0 - alpha
# This function helps to add two image with specific weight parameter to get a third image as summation of two image.
img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
(thresh, img_final_bin) = cv2.threshold(img_final_bin, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#cv2.imwrite("img_final_bin.jpg",img_final_bin)

In [ ]:
# definig this code from https://www.pyimagesearch.com/2015/04/20/sorting-contours-using-python-and-opencv/
def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0
    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

In [ ]:
# Find contours for image, which will detect all the boxes
contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#im2, contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Sort all the contours by top to bottom.
(contours, boundingBoxes) = sort_contours(contours, method="top-to-bottom")

# Croping the character and store into the croped folder

In [ ]:
croped_dir_path = "croped/cropA"
idx = 0
oth = 0
for c in contours:
    # Returns the location and width,height for every contour
    x, y, w, h = cv2.boundingRect(c)
    if (290 > w > 270 and  150 > h > 140):
        idx += 1
        new_img = img[y:y+h, x:x+w]
        cv2.imwrite(croped_dir_path+str(idx) + '.png', new_img)
    else:
        oth += 1
        new_img = img[y:y+h, x:x+w]
        cv2.imwrite(croped_dir_path+"_not_used"+str(oth) + '.png', new_img)

# Removing crop_not_used images

In [ ]:
if input("Enter yes to remove unwaned files: ").lower() == "yes":
    for i in os.listdir("croped"):
        if "crop_not_used" in i:
            os.remove("croped/"+i)
            print("Removed File: " + i)

Enter yes to remove unwaned files: n


# Removing unwanted Space in croped images

In [ ]:
for i in os.listdir("croped"):
    img = cv2.imread("croped/"+i)
    blurred = cv2.blur(img, (6,6))
    canny = cv2.Canny(blurred, 50, 200)
    ## find the non-zero min-max coords of canny
    pts = np.argwhere(canny>0)
    y1,x1 = pts.min(axis=0)
    y2,x2 = pts.max(axis=0)
    ## crop the region
    cropped = img[y1:y2, x1:x2]
    cv2.imwrite("croped/"+i, cropped)

# Crop Specific image if needed ->> press C to crop  press esc to exit

In [ ]:
img_path = 'croped/crop29.png'
image = cv2.imread(img_path)
image_to_show = np.copy(image)
mouse_pressed = False
s_x = s_y = e_x = e_y = -1
def mouse_callback(event, x, y, flags, param):
    global image_to_show, s_x, s_y, e_x, e_y, mouse_pressed
    if event == cv2.EVENT_LBUTTONDOWN:
        mouse_pressed = True
        s_x, s_y = x, y
        image_to_show = np.copy(image)
    elif event == cv2.EVENT_MOUSEMOVE:
        if mouse_pressed:
            image_to_show = np.copy(image)
            cv2.rectangle(image_to_show, (s_x, s_y),(x, y), (0, 255, 0), 1)
    elif event == cv2.EVENT_LBUTTONUP:
        mouse_pressed = False
        e_x, e_y = x, y
cv2.namedWindow('image')
cv2.setMouseCallback('image', mouse_callback)
while True:
    cv2.imshow('image', image_to_show)
    k = cv2.waitKey(1)
    if k == ord('c'):
        if s_y > e_y:s_y, e_y = e_y, s_y
        if s_x > e_x:s_x, e_x = e_x, s_x
        if e_y - s_y > 1 and e_x - s_x > 0:
            image = image[s_y:e_y, s_x:e_x]
            image_to_show = np.copy(image)
    elif k == 27:break
cv2.destroyAllWindows()
cv2.imwrite(img_path,image_to_show)

# Image conversion

In [ ]:
a = cv2.imread("croped/crop1.png")
a_gray = cv2.cvtColor(a, cv2.COLOR_BGR2)
a_blur = cv2.blur(a_gray, (15,15))
plt.imshow(a_gray)

# Characters Recognition

In [ ]:
img_rgb = cv2.imread("handwritten Characters/Preyash/1.jpg")
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
img_blur = cv2.blur(img_gray, (15,15))
for i in os.listdir("croped"):
    template = cv2.imread("croped/"+i,0)
    w, h = template.shape[::-1]
    res = cv2.matchTemplate(img_blur,template,cv2.TM_CCOEFF_NORMED)
    threshold = 0.8
    loc = np.where( res >= threshold)
    for pt in zip(*loc[::-1]):
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
        cv2.putText(img_rgb, i, (pt[0] + w, pt[1] + h),cv2.FONT_HERSHEY_SIMPLEX , 1,(255,0,0))
cv2.imwrite('output.png',img_rgb)